## Optimization

This is the notebook for running optimization to solve the tree siting problem in Center City District, Philadelphia. The method used is called Non-dominated Sorting Genetic Algorithm II (NSGA-II), which is a popular multi-objective optimization algorithm and proved as a very validated and successful method. It is implemented using the `pymoo` library, which provides a convenient interface for multi-objective optimization algorithms. It does not require a predefined weighting parameter for the objectives, and generates a diverse set of solutions around pareto front.

Our two objectives are:

$$
\text{Z}_{shade} = \sum_i S_i x_i - \sum_{i < j} O_{ij} x_i x_j
$$
$$
\text{Z}_{coolwalk} ^ \alpha = 
\frac{
    \displaystyle\sum_{m \in V_o,\, n \in V_{d}(m)} 
    w_{mn} \left[
        \Lambda_{m \rightarrow n}^{\alpha, *}(0)
        - \Lambda_{m \rightarrow n}^{\alpha, *}(\mathcal{T})
    \right]
}{
    \displaystyle\sum_{m \in V_o,\, n \in V_{d}(m)}
    w_{mn} \left[
        \Lambda_{m \rightarrow n}^{\alpha, *}(0)
        - \Lambda_{m \rightarrow n}^{\alpha, *}(1)
    \right]
}
$$
$$
\lambda_m = \lambda_m^{shaded} + \alpha \times \lambda_m^{sun}
$$



where: 

- $i, j$ are the potential tree sites
- $S_i$ is the unique shade area of tree site $i$
- $O_{ij}$ is the overlap area between tree sites $i$ and $j$

- $\alpha$ is the weighting parameter for the not shaded length of an edge on the graph
- $\lambda_m$ is the updated length of an edge $m$ after shading
- $\lambda_m^{shaded}$ is the shaded length of an edge $m$
- $\lambda_m^{sun}$ is the unshaded length of an edge $m$
- $V_o$ is the set of origin nodes
- $V_d(m)$ is the set of destination nodes for origin node $m$
- $w_{mn}$ is the traffic weight of the OD pair $(m, n)$
- $\Lambda_{m \rightarrow n}^{\alpha, *}(0)$ is the shortest path length from $m$ to $n$ when edges are fully unshaded (i.e., no shaded length)
- $\Lambda_{m \rightarrow n}^{\alpha, *}(1)$ is the shortest path length from $m$ to $n$ when edges are fully shaded (i.e., all shaded length)
- $\Lambda_{m \rightarrow n}^{\alpha, *}(\mathcal{T})$ is the shortest path length from $m$ to $n$ when edges are shaded by a selected set of new trees $\mathcal{T}$

We have run the optimization for three different values of $\alpha$: 1.01, 2.0, and 100.0, which represent different levels of perceptions of the unshaded walking experience. We also tested the optimization with two different tree selection sizes: 500 and 1000. The study area is limited to Center City District, Philadelphia, and sampled the 44,818 OD pairs from 158,652 pairs, due to the capability of the PC.

It took about 6 hours to run the optimization tasks for 6 scenarios in total, when pop_size=30 (larger can produce more accurate results but raise resource cost) and 8 CPU cores in parallel. 

In [6]:
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import geopandas as gpd


tree_gdf = gpd.read_file('data/final/tree_sites_all.geojson')
ccd_gdf = gpd.read_file('data/final/CCD_BOUNDARY.geojson').to_crs(tree_gdf.crs)

ccd_tree_gdf = gpd.sjoin(tree_gdf, ccd_gdf, how='inner', predicate='within')
tree_ids = ccd_tree_gdf['original_index'].tolist()
S = dict(zip(tree_ids, ccd_tree_gdf['unique_shade_area'].tolist()))


overlap_df = pd.read_csv('data/final/tree_pairwise_overlap.csv')
ccd_overlap_df = overlap_df.loc[overlap_df['i'].isin(tree_ids) & overlap_df['j'].isin(tree_ids)]
O = {(int(r['i']), int(r['j'])): float(r['overlap_area']) for _, r in ccd_overlap_df.iterrows()}
O.update({(j, i): a for (i, j), a in O.items()})  # symmetric

# O ={(i, j): O[(i, j)] for (i, j) in O if i in tree_ids_set and j in tree_ids_set}

shade_edge_df = pd.read_csv('data/final/shade_on_edge.csv')
ccd_shade_edge_df = shade_edge_df.loc[shade_edge_df['site_id'].isin(tree_ids)]
shade_on_edge = {}
for _, r in ccd_shade_edge_df.iterrows():
    t, e, L = int(r['site_id']), int(r['edge_id']), float(r['shade_length'])
    shade_on_edge.setdefault(t, {})[e] = L

# shade_on_edge = {t: shade_on_edge[t] for t in tree_ids if t in shade_on_edge}

overlap_edge_df = pd.read_csv('data/final/overlap_shade_on_edge.csv')
ccd_overlap_edge_df = overlap_edge_df.loc[overlap_edge_df['site_i'].isin(tree_ids) & overlap_edge_df['site_j'].isin(tree_ids)]
overlap_shade_on_edge = {}
for _, r in ccd_overlap_edge_df.iterrows():
    i, j, e, L = int(r['site_i']), int(r['site_j']), int(r['edge_id']), float(r['overlap_shade_length'])
    key = (i, j) if i < j else (j, i)
    overlap_shade_on_edge.setdefault(key, {})[e] = L

# overlap_shade_on_edge = {
#     (i, j): overlap_shade_on_edge[(i, j)]
#     for (i, j) in overlap_shade_on_edge if i in tree_ids_set and j in tree_ids_set
# }

import geopandas as gpd
from shapely.geometry import Point

# load data
with open('data/foot_traffic/G.pkl', 'rb') as f:
    G = pickle.load(f)

od_base_df = pd.read_csv('data/foot_traffic/od_results_with_W.csv')
od_top5_df = pd.read_pickle('data/foot_traffic/od_top5_shortest_paths.pkl')

# Step 1: node coords
node_records = []
for n in G.nodes:
    x = G.nodes[n]['x']
    y = G.nodes[n]['y']
    node_records.append({'node_id': n, 'geometry': Point(x, y)})

node_gdf = gpd.GeoDataFrame(node_records, crs=tree_gdf.crs)

# Step 2: get all nodes in CCD polygon
ccd_nodes_gdf = gpd.sjoin(node_gdf, ccd_gdf, how='inner', predicate='within')
ccd_nodes = set(ccd_nodes_gdf['node_id'])

ccd_od_base_df = od_base_df[
    od_base_df['origin_node'].isin(ccd_nodes) &
    od_base_df['dest_node'].isin(ccd_nodes)
].copy()

ccd_od_top5_df = od_top5_df[
    od_top5_df['origin_node'].isin(ccd_nodes) &
    od_top5_df['dest_node'].isin(ccd_nodes)
].copy()


edge_id_map = {data["edge_id"]: (u, v) for u, v, data in G.edges(data=True)}


In [8]:
len(ccd_tree_gdf)

4677

In [2]:
import time
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.sampling import Sampling
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.operators.crossover.pntx import TwoPointCrossover as BinaryTwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation as BinaryBitflipMutation
from pymoo.termination.ftol import MultiObjectiveSpaceTermination
from pymoo.termination.robust import RobustTermination
from pymoo.termination.max_gen import MaximumGenerationTermination
from pymoo.termination.default import DefaultMultiObjectiveTermination
from multiprocessing.pool import ThreadPool
from pymoo.core.problem import StarmapParallelization
import multiprocessing
import numpy as np

# ==== 目标函数实现 ====
def compute_Z_shade(selected):
    sel = list(selected)
    shade = sum(S[i] for i in sel)
    overlap = sum(O.get((sel[i], sel[j]), 0.0) for i in range(len(sel)) for j in range(i+1, len(sel)))
    return shade - overlap

def compute_edge_shade(selected):
    edge_shade = {}
    for t in selected:
        for e, L in shade_on_edge.get(t, {}).items():
            edge_shade[e] = edge_shade.get(e, 0.0) + L
    sel = list(selected)
    for i in range(len(sel)):
        for j in range(i+1, len(sel)):
            key = (sel[i], sel[j]) if sel[i] < sel[j] else (sel[j], sel[i])
            for e, L in overlap_shade_on_edge.get(key, {}).items():
                edge_shade[e] = edge_shade.get(e, 0.0) - L
    return edge_shade

def compute_Z_coolwalk(selected, od_base=ccd_od_base_df, od_top5=ccd_od_top5_df, alpha=2.0):
    edge_shade = compute_edge_shade(selected)
    edge_lambda = {}
    for eid, (u, v) in edge_id_map.items():
        l = G[u][v]['length']
        shaded = min(edge_shade.get(eid, 0), l)
        lambda_e = alpha * max(l - shaded, 0) + shaded
        edge_lambda[eid] = max(lambda_e, 0.01)
    total = 0
    for _, row in od_top5.iterrows():
        lengths = []
        for i in range(1, 6):
            path = row[f'top_{i}_shortest']
            if not path:
                lengths.append(np.inf)
            else:
                lengths.append(sum(edge_lambda.get(eid, 1e6) for eid in path))
        best_len = min(lengths)
        o, d = int(row.origin_node), int(row.dest_node)
        traffic_row = od_base.query("origin_node == @o and dest_node == @d")
        if not traffic_row.empty:
            weight = float(traffic_row['traffic_o'].values[0]) * float(traffic_row['W'].values[0])
            total += best_len * weight
    full_shade = np.sum(od_base['path_length'] * od_base['traffic_o'] * od_base['W'])
    no_shade = alpha * full_shade
    return (no_shade - total) / (no_shade - full_shade)

# ==== 多目标优化Problem ====
class TreeParetoProblemCCD(ElementwiseProblem):
    def __init__(self, tree_ids, S, O, shade_on_edge, overlap_shade_on_edge,
                 G, edge_id_map, od_top5_df, od_base_df, N, alpha=2.0, elementwise_runner=None, **kwargs):
        super().__init__(n_var=len(tree_ids), n_obj=2, n_constr=1, xl=0, xu=1, vtype=int, 
                         elementwise_runner=elementwise_runner, **kwargs)
        self.tree_ids = tree_ids
        self.S = S
        self.O = O
        self.shade_on_edge = shade_on_edge
        self.overlap_shade_on_edge = overlap_shade_on_edge
        self.G = G
        self.edge_id_map = edge_id_map
        self.od_top5_df = od_top5_df
        self.od_base_df = od_base_df
        self.N = N
        self.alpha = alpha

    def _evaluate(self, x, out, *args, **kwargs):
        sel = [self.tree_ids[i] for i, xi in enumerate(x) if xi == 1]
        out["G"] = [abs(len(sel) - self.N)]
        if len(sel) != self.N:
            out["F"] = [1e6, 1e6]
            return
        z_shade = -compute_Z_shade(sel)
        z_coolwalk = -compute_Z_coolwalk(sel, od_base=self.od_base_df, od_top5=self.od_top5_df, alpha=self.alpha)
        out["F"] = [z_shade, z_coolwalk]

# ==== 固定选N棵树的采样方法 ====
class FixedSumBinarySampling(Sampling):
    def __init__(self, N):
        super().__init__()
        self.N = N
    def _do(self, problem, n_samples, **kwargs):
        X = []
        for _ in range(n_samples):
            arr = np.zeros(problem.n_var, dtype=int)
            idx = np.random.choice(problem.n_var, self.N, replace=False)
            arr[idx] = 1
            X.append(arr)
        return np.array(X)

# ==== 推荐: CCD数据分层采样（防止爆内存） ====
def stratified_od_sampling(od_base_df, od_top5_df, sample_ratio=0.15):
    print(f"Original number of OD pairs in CCD: {len(od_base_df)}")
    od_with_weight = od_base_df.copy()
    od_with_weight['weight'] = od_with_weight['traffic_o'] * od_with_weight['W']
    q33 = od_with_weight['weight'].quantile(0.33)
    q67 = od_with_weight['weight'].quantile(0.67)
    high_weight = od_with_weight[od_with_weight['weight'] > q67]
    mid_weight = od_with_weight[(od_with_weight['weight'] > q33) & (od_with_weight['weight'] <= q67)]
    low_weight = od_with_weight[od_with_weight['weight'] <= q33]
    n_high = int(len(high_weight) * 0.6)
    n_mid = int(len(mid_weight) * 0.2)
    n_low = int(len(low_weight) * 0.05)
    sampled_high = high_weight.sample(n=min(n_high, len(high_weight)), random_state=42)
    sampled_mid = mid_weight.sample(n=min(n_mid, len(mid_weight)), random_state=42)
    sampled_low = low_weight.sample(n=min(n_low, len(low_weight)), random_state=42)
    sampled_od_base = pd.concat([sampled_high, sampled_mid, sampled_low])
    sampled_pairs = set(zip(sampled_od_base['origin_node'], sampled_od_base['dest_node']))
    mask = od_top5_df.apply(lambda row: (row['origin_node'], row['dest_node']) in sampled_pairs, axis=1)
    sampled_od_top5 = od_top5_df[mask].reset_index(drop=True)
    print(f"Sampled number of OD pairs in CCD: {len(sampled_od_base)}")
    print(f"Sampled number of OD pairs' top 5 routes in CCD: {len(sampled_od_top5)}")
    return sampled_od_base.reset_index(drop=True), sampled_od_top5

# ==== 运行多目标NSGA-II ====
if __name__ == "__main__":
    alpha_list = [1.01, 2.0, 100.0]
    # alpha_list = [2.0, 100.0]
    N_select_list = [1000, 500]
    MAX_GENERATIONS = 50

    # Initialize process pool for parallelization (better for CPU-intensive tasks)
    n_processes = 8  # Adjust based on your CPU cores
    pool = multiprocessing.Pool(n_processes)
    runner = StarmapParallelization(pool.starmap)

    try:
        for ALPHA in alpha_list:
            for N_SELECT in N_select_list:
                print(f"Running NSGA-II with ALPHA={ALPHA}, N_SELECT={N_SELECT}")
                # 步骤1：采样CCD范围OD对
                sampled_od_base, sampled_od_top5 = stratified_od_sampling(ccd_od_base_df, ccd_od_top5_df, sample_ratio=0.1)

                # 步骤2：NSGA-II优化
                problem = TreeParetoProblemCCD(
                    tree_ids=tree_ids,
                    S=S, O=O,
                    shade_on_edge=shade_on_edge, overlap_shade_on_edge=overlap_shade_on_edge,
                    G=G, edge_id_map=edge_id_map,
                    od_top5_df=sampled_od_top5, od_base_df=sampled_od_base,
                    N=N_SELECT, alpha=ALPHA,
                    elementwise_runner=runner
                )
                algorithm = NSGA2(
                    pop_size=30,
                    sampling=FixedSumBinarySampling(N_SELECT),
                    crossover=BinaryTwoPointCrossover(),
                    mutation=BinaryBitflipMutation(),
                    eliminate_duplicates=True
                )
                # termination_gen = get_termination("n_gen", MAX_GENERATIONS)
                

                # termination_robust = RobustTermination(
                #     MultiObjectiveSpaceTermination(tol=1e-6, n_skip=10),
                #     period=5
                # )

                # termination = termination_robust | termination_gen

                # termination = RobustTermination(
                #     MultiObjectiveSpaceTermination(tol=1e-6, n_skip=10),
                #     period=5
                # )
        

                termination = DefaultMultiObjectiveTermination(
                    xtol=0,
                    cvtol=0,
                    ftol=1e-6,
                    period=5,
                    n_max_gen=MAX_GENERATIONS,
                    n_max_evals=np.inf
                )

                t0 = time.time()
                res = minimize(problem, algorithm, termination, seed=42, verbose=True)
                t1 = time.time()
                print(f"NSGA-II solve time: {t1-t0:.2f}s")

                pareto_solutions = []
                for i, x in enumerate(res.X):
                    sel = [tree_ids[j] for j, xi in enumerate(x) if xi == 1]
                    Z_shade = -res.F[i, 0]
                    Z_coolwalk = -res.F[i, 1]
                    pareto_solutions.append({
                        "tree_ids": sel,
                        "Z_shade": Z_shade,
                        "Z_coolwalk": Z_coolwalk
                    })
                with open(f"data/optimization/ccd_pareto_solutions_{ALPHA*100:.0f}_{N_SELECT}.pkl", "wb") as f:
                    pickle.dump({
                        'pareto_solutions': pareto_solutions,
                        'n_selected': N_SELECT,
                        'n_candidates': len(tree_ids),
                        'n_od_sampled': len(sampled_od_base),
                        'n_odtop5_sampled': len(sampled_od_top5),
                        'alpha': ALPHA,
                        'solve_time': t1-t0
                    }, f)
                print(f"Saved ccd_pareto_solutions_{ALPHA*100:.0f}_{N_SELECT}.pkl")

    finally:
        # Clean up the thread pool
        pool.close()
        pool.join()


Running NSGA-II with ALPHA=1.01, N_SELECT=1000
Original number of OD pairs in CCD: 158652
Sampled number of OD pairs in CCD: 44818
Sampled number of OD pairs' top 5 routes in CCD: 44818
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       30 |      4 |  0.000000E+00 |  0.000000E+00 |             - |             -
     2 |       60 |      4 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     3 |       90 |      5 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     4 |      120 |      6 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     5 |      150 |      8 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     6 |      180 |     10 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     7 |      210 |      3 |  0.000000E+00 |  0.000000E+00 |  0.0842624917 |         ideal
     8 |      240 |      4 |  0.000000E+00 |  0.000000E+00 |  0.0040293435 |          

Below is the code to run optimization without sampling the OD pairs. 

In [ ]:
def compute_Z_shade(selected):
    sel = list(selected)
    shade = sum(S[i] for i in sel)
    overlap = sum(O.get((sel[i], sel[j]), 0.0) for i in range(len(sel)) for j in range(i+1, len(sel)))
    return shade - overlap

def compute_edge_shade(selected):
    edge_shade = {}
    for t in selected:
        for e, L in shade_on_edge.get(t, {}).items():
            edge_shade[e] = edge_shade.get(e, 0.0) + L
    sel = list(selected)
    for i in range(len(sel)):
        for j in range(i+1, len(sel)):
            key = (sel[i], sel[j]) if sel[i] < sel[j] else (sel[j], sel[i])
            for e, L in overlap_shade_on_edge.get(key, {}).items():
                edge_shade[e] = edge_shade.get(e, 0.0) - L
    return edge_shade

def compute_Z_coolwalk(selected, alpha=2.0):
    edge_shade = compute_edge_shade(selected)
    edge_lambda = {}
    for eid, (u, v) in edge_id_map.items():
        l = G[u][v]['length']
        shaded = min(edge_shade.get(eid, 0), l)
        lambda_e = alpha * max(l - shaded, 0) + shaded
        edge_lambda[eid] = max(lambda_e, 0.01)
    total = 0
    for _, row in od_top5_df.iterrows():
        lengths = []
        for i in range(1, 6):
            path = row[f'top_{i}_shortest']
            if not path:
                lengths.append(np.inf)
            else:
                lengths.append(sum(edge_lambda.get(eid, 1e6) for eid in path))
        best_len = min(lengths)
        o, d = int(row.origin_node), int(row.dest_node)
        traffic_row = ccd_od_base_df.query("origin_node == @o and dest_node == @d")
        if not traffic_row.empty:
            weight = float(traffic_row['traffic_o'].values[0]) * float(traffic_row['W'].values[0])
            total += best_len * weight
    full_shade = np.sum(ccd_od_base_df['path_length'] * ccd_od_base_df['traffic_o'] * ccd_od_base_df['W'])
    no_shade = alpha * full_shade
    return (no_shade - total) / (no_shade - full_shade)


from pymoo.core.problem import ElementwiseProblem

class TreeParetoProblem(ElementwiseProblem):
    def __init__(self, tree_ids, S, O, shade_on_edge, overlap_shade_on_edge, G, edge_id_map, od_top5_df, od_base_df, N, alpha=2.0):
        super().__init__(n_var=len(tree_ids), n_obj=2, n_constr=1, xl=0, xu=1, vtype=int)
        self.tree_ids = tree_ids
        self.S = S
        self.O = O
        self.shade_on_edge = shade_on_edge
        self.overlap_shade_on_edge = overlap_shade_on_edge
        self.G = G
        self.edge_id_map = edge_id_map
        self.od_top5_df = ccd_od_top5_df
        self.od_base_df = ccd_od_base_df
        self.N = N
        self.alpha = alpha

    def _evaluate(self, x, out, *args, **kwargs):
        sel = [self.tree_ids[i] for i, xi in enumerate(x) if xi == 1]
        out["G"] = [abs(len(sel) - self.N)]
        if len(sel) != self.N:
            out["F"] = [1e6, 1e6]
            return
        z_shade = -compute_Z_shade(sel)
        z_coolwalk = -compute_Z_coolwalk(sel, self.alpha)
        out["F"] = [z_shade, z_coolwalk]

from pymoo.core.sampling import Sampling
import numpy as np

class FixedSumBinarySampling(Sampling):
    def __init__(self, N):
        super().__init__()
        self.N = N
    def _do(self, problem, n_samples, **kwargs):
        X = []
        for _ in range(n_samples):
            arr = np.zeros(problem.n_var, dtype=int)
            idx = np.random.choice(problem.n_var, self.N, replace=False)
            arr[idx] = 1
            X.append(arr)
        return np.array(X)
    

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.termination import get_termination

from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.crossover.pntx import TwoPointCrossover as BinaryTwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation as BinaryBitflipMutation


N_SELECT = 10

problem = TreeParetoProblem(
    tree_ids=tree_ids,
    S=S, O=O,
    shade_on_edge=shade_on_edge, overlap_shade_on_edge=overlap_shade_on_edge,
    G=G, edge_id_map=edge_id_map,
    od_top5_df=od_top5_df, od_base_df=od_base_df,
    N=N_SELECT, alpha=2.0
)

algorithm = NSGA2(
    pop_size=30,
    sampling=FixedSumBinarySampling(N_SELECT),
    crossover=BinaryTwoPointCrossover(),
    mutation=BinaryBitflipMutation(),
    eliminate_duplicates=True
)
termination = get_termination("n_gen", 30)

import time
t0 = time.time()
res = minimize(problem, algorithm, termination, seed=42, verbose=True)
t1 = time.time()
print(f"NSGA-II solve time: {t1-t0:.2f}s")

pareto_solutions = []
for i, x in enumerate(res.X):
    sel = [tree_ids[j] for j, xi in enumerate(x) if xi == 1]
    Z_shade = -res.F[i, 0]
    Z_coolwalk = -res.F[i, 1]
    pareto_solutions.append({
        "tree_ids": sel,
        "Z_shade": Z_shade,
        "Z_coolwalk": Z_coolwalk
    })


Randomly selecting results comparison.

Compared to the random selection results, the optimization results significantly improved the shaded area by 1.16-7.30%, and the coolwalkability by 89.76-205.43%.

In [14]:
import numpy as np
import pandas as pd
import pickle
from tqdm import trange

alpha_list = [1.01, 2.0, 100.0]
N_select_list = [1000, 500]
N_REPEAT = 100  # 随机实验次数

results = []

for ALPHA in alpha_list:
    for N_SELECT in N_select_list:
        print(f"\n==== α={ALPHA}, N={N_SELECT} ====")
        # 加载优化结果
        fname = f"data/optimization/ccd_pareto_solutions_{ALPHA*100:.0f}_{N_SELECT}.pkl"
        with open(fname, "rb") as f:
            data = pickle.load(f)
        pareto_solutions = data['pareto_solutions']

        # ======= 归一化并选最接近理想点解 =======
        zs = np.array([[s['Z_shade'], s['Z_coolwalk']] for s in pareto_solutions])
        # 防止出现常数列导致除零
        zs_min = zs.min(axis=0)
        zs_max = zs.max(axis=0)
        zs_norm = (zs - zs_min) / np.where(zs_max - zs_min == 0, 1, (zs_max - zs_min))
        ideal = np.ones(2)  # 归一化后理想点(1,1)
        dist = np.linalg.norm(zs_norm - ideal, axis=1)
        best_idx = np.argmin(dist)
        best_solution = pareto_solutions[best_idx]
        opt_z_shade = best_solution['Z_shade']
        opt_z_coolwalk = best_solution['Z_coolwalk']

        # ======= 随机采样 N_REPEAT 次，统计均值和std =======
        rand_results = []
        for _ in trange(N_REPEAT, desc=f"Random α={ALPHA},N={N_SELECT}", leave=False):
            rand_sel = np.random.choice(tree_ids, N_SELECT, replace=False)
            z_shade = compute_Z_shade(rand_sel)
            z_coolwalk = compute_Z_coolwalk(rand_sel, od_base=sampled_od_base, od_top5=sampled_od_top5, alpha=ALPHA)
            rand_results.append({'Z_shade': z_shade, 'Z_coolwalk': z_coolwalk})
        rand_shade = np.array([r['Z_shade'] for r in rand_results])
        rand_coolwalk = np.array([r['Z_coolwalk'] for r in rand_results])

        # ======= 计算提升 =======
        improve_shade = (opt_z_shade - rand_shade.mean()) / rand_shade.mean() * 100
        improve_coolwalk = (opt_z_coolwalk - rand_coolwalk.mean()) / rand_coolwalk.mean() * 100

        # ======= 结果存入表格 =======
        results.append({
            'alpha': ALPHA,
            'N_select': N_SELECT,
            'opt_Z_shade': opt_z_shade,
            'rand_Z_shade_mean': rand_shade.mean(),
            'rand_Z_shade_std': rand_shade.std(),
            'shade_improve_%': improve_shade,
            'opt_Z_coolwalk': opt_z_coolwalk,
            'rand_Z_coolwalk_mean': rand_coolwalk.mean(),
            'rand_Z_coolwalk_std': rand_coolwalk.std(),
            'coolwalk_improve_%': improve_coolwalk,
        })

# ======= 保存为CSV =======
df = pd.DataFrame(results)
df.to_csv("optimization_vs_random_compare_ideal.csv", index=False)
print(df)
print("\nSaved as optimization_vs_random_compare_ideal.csv")



==== α=1.01, N=1000 ====



==== α=1.01, N=500 ====



==== α=2.0, N=1000 ====



==== α=2.0, N=500 ====



==== α=100.0, N=1000 ====



==== α=100.0, N=500 ====


    alpha  N_select  opt_Z_shade  rand_Z_shade_mean  rand_Z_shade_std  \
0    1.01      1000     696998.0          649608.40      12805.284802   
1    1.01       500     342169.0          335788.90       9737.272287   
2    2.00      1000     657983.0          650465.46      12875.042158   
3    2.00       500     353334.0          334520.57      11642.925183   
4  100.00      1000     657983.0          649501.58      13585.078506   
5  100.00       500     351422.0          334548.62       9777.317705   

   shade_improve_%  opt_Z_coolwalk  rand_Z_coolwalk_mean  rand_Z_coolwalk_std  \
0         7.295103        0.001081              0.000507             0.000330   
1         1.900033        0.000902              0.000295             0.000297   
2         1.155717        0.001415              0.000719             0.000462   
3         5.623998        0.001169              0.000423             0.000409   
4         1.305835        0.001471              0.000775             0.000489   
5 